**Задача 1. Необходимо составить и отсортировать сводную таблицу и вывести среднюю сумму инвестиций для некоторых стран. Данные за каждый год должны быть в отдельном поле.** 

<pre>WITH y2011 AS   
(SELECT country_code,  
       AVG(funding_total) AS avg_11  
FROM company  
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = 2011  
GROUP BY country_code),  
     y2012 AS   
(SELECT country_code,  
       AVG(funding_total) AS avg_12  
FROM company  
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = 2012  
GROUP BY country_code),  
     y2013 AS   
(SELECT country_code,  
       AVG(funding_total) AS avg_13  
FROM company  
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) = 2013  
GROUP BY country_code)  

SELECT y11.country_code,  
       avg_11,  
       avg_12,  
       avg_13  
FROM y2011 AS y11  
INNER JOIN y2012 AS y12 ON y11.country_code = y12.country_code  
INNER JOIN y2013 AS y13 ON y12.country_code = y13.country_code  
ORDER BY avg_11 DESC<pre>  

**Задача 2. Необходимо отобрать и сгруппировать данные по условию за определенный период.** 

<pre>WITH acquis AS 
  (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS timestamp)) AS mon_acq,
          COUNT(id) AS cou,
          SUM(price_amount) AS price
   FROM acquisition
   WHERE EXTRACT(YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN '2010' AND '2013'
   GROUP BY EXTRACT(MONTH FROM CAST(acquired_at AS timestamp))),

  fun_name AS
  (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)) AS mon_fun,
          COUNT(DISTINCT f.name) AS name_fun
   FROM fund AS f 
   INNER JOIN investment AS i ON f.id = i.fund_id     
   INNER JOIN funding_round AS fr ON  i.funding_round_id = fr.id
   INNER JOIN company AS c ON fr.company_id = c.id
WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN '2010' AND '2013'
  AND f.country_code = 'USA'
GROUP BY EXTRACT(MONTH FROM CAST(fr.funded_at AS timestamp)))

SELECT f.mon_fun,
       f.name_fun,
       a.cou,
       a.price
FROM fun_name AS f 
INNER JOIN acquis AS a ON f.mon_fun = a.mon_acq<pre>     


**Задача 3. Необходимо выгрузить таблицу, с названиями компаний определенной категории, по условию за определнный период.**

<pre>SELECT c.name,
       EXTRACT(MONTH FROM CAST(f.funded_at AS timestamp))
FROM company AS c
INNER JOIN funding_round AS f ON c.id = f.company_id
WHERE c.category_code = 'social'
  AND f.raised_amount !=0
  AND EXTRACT(YEAR FROM CAST(f.funded_at AS timestamp)) BETWEEN '2010' AND '2013'<pre>

**Задача 4. Необходимо выгрузить и отсортировать таблицу с определенными полями, добавив новый столбец и исключив объекты по условию.**

<pre>SELECT cing.name,
       a.price_amount,
       cer.name,
       cer.funding_total,
       ROUND(a.price_amount/cer.funding_total)
FROM company AS cing
INNER JOIN acquisition AS a ON cing.id = a.acquiring_company_id
INNER JOIN company AS cer ON a.acquired_company_id = cer.id
WHERE a.price_amount !=0
  AND cer.funding_total !=0
ORDER BY a.price_amount DESC, cer.name
LIMIT 10<pre>


**Задача 5. Необходимо составить таблицу из определенных полей, за заданный период и по условию.**

<pre>SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM fund AS f
INNER JOIN investment AS i ON f.id = i.fund_id
INNER JOIN funding_round AS fr ON i.funding_round_id = fr.id
INNER JOIN company AS c ON fr.company_id = c.id
WHERE c.milestones > 6
  AND EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN '2012' AND '2013'<pre>
  

**Задача 6. Необходимо вывести среднее число учебных заведений по условию.**

<pre>WITH instit AS
(SELECT p.id, 
        COUNT(e.instituition) AS coun
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id

WHERE company_id IN

(SELECT id AS company_id
FROM company
WHERE name = 'Facebook')
AND e.instituition IS NOT NULL
GROUP BY p.id)

SELECT AVG(coun)
FROM instit<pre>

**Задача 7. Необходимо вывести среднее число учебных заведений по заданному условию.**

<pre>WITH instit AS
(SELECT p.id, 
       COUNT(e.instituition) AS coun
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id

WHERE company_id IN

(SELECT id AS copany_id
FROM company
WHERE id IN (SELECT company_id AS id
             FROM funding_round
             WHERE is_first_round = 1
               AND is_last_round = 1)
AND status = 'closed')
AND e.instituition IS NOT NULL
GROUP BY p.id)

SELECT AVG(coun)
FROM instit<pre>